In [1]:
# !pip install antropy
# !git clone https://github.com/raphaelvallat/antropy/

In [2]:
# !python.exe -m pip install --upgrade pip

In [32]:
# Imports
import antropy as ant
import numpy as np
import pandas as pd
import math
# from antropy.antropy import entropy 
# from scipy.stats import entropy
import scipy.signal
# from envelope_derivative_operator.energy_operators import edo
from scipy.signal import periodogram, welch
from math import factorial, log
from sklearn.preprocessing import StandardScaler

In [33]:
# Install necessary git repositories and pip installations

!git clone https://github.com/ktapani/Neonatal_Seizure_Detection.git
!git clone https://github.com/raphaelvallat/antropy.git
!git clone https://github.com/otoolej/envelope_derivative_operator.git
!pip install numba


fatal: destination path 'Neonatal_Seizure_Detection' already exists and is not an empty directory.
fatal: destination path 'antropy' already exists and is not an empty directory.
fatal: destination path 'envelope_derivative_operator' already exists and is not an empty directory.


In [9]:
%%time
# Read your respective window lengths file

df = pd.read_csv('filtered_babydf8sec.csv')

CPU times: total: 54.1 s
Wall time: 1min 8s


In [10]:
df

,0,1,2,3,4,5,6,7,8,9,...,5367,5368,5369,5370,5371,5372,5373,5374,5375,5376
0,0.675267,0.000000,0.290726,0.230747,0.495942,0.495182,0.665196,0.581555,0.734356,0.714433,...,0.657762,0.613065,0.569667,0.527994,0.501359,0.480403,0.465786,0.456672,0.459519,0.0
1,0.431357,0.454031,0.400183,0.368709,0.297342,0.312396,0.376014,0.486442,0.531480,0.538681,...,0.253163,0.249772,0.247816,0.248635,0.252143,0.257809,0.267183,0.279109,0.292968,0.0
2,0.470399,0.439181,0.516854,0.476959,0.413500,0.292791,0.436052,0.130145,0.362178,0.153259,...,0.656448,0.638892,0.594614,0.548856,0.498811,0.459506,0.449605,0.472397,0.519079,0.0
3,0.571449,0.607684,0.590456,0.608173,0.520397,0.416818,0.318117,0.047866,0.235091,0.106873,...,0.467948,0.500905,0.512297,0.522532,0.510332,0.493605,0.470348,0.465390,0.439050,0.0
4,0.563856,0.450437,0.533854,0.541654,0.546351,0.230296,0.263740,0.114176,0.000000,0.119783,...,0.272985,0.261871,0.254642,0.255609,0.257484,0.275836,0.293726,0.328193,0.365759,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47871,0.409119,0.314726,0.377140,0.580123,0.721367,0.805053,0.809710,0.599189,0.347885,0.192095,...,0.386580,0.445764,0.414810,0.399375,0.436431,0.488679,0.532678,0.578072,0.496491,0.0
47872,0.607278,0.427792,0.390650,0.615235,0.886784,0.911227,0.875492,0.828515,0.792090,0.761711,...,0.661060,0.643501,0.695311,0.714864,0.716173,0.658541,0.644344,0.659286,0.679108,0.0
47873,0.610637,0.578028,0.578676,0.563010,0.572841,0.541571,0.517731,0.509364,0.499299,0.503451,...,0.722861,0.752414,0.720291,0.704067,0.719866,0.703545,0.690849,0.661180,0.667561,0.0
47874,0.415536,0.540711,0.423628,0.523628,0.509501,0.470432,0.472035,0.510317,0.467849,0.414331,...,0.726438,0.612115,0.531106,0.420183,0.478467,0.440619,0.583377,0.511047,0.458242,0.0


<ins>Features considered in this notebook:</ins>

Total number of features considered = 11

- Mean
- Standard Deviation
- Kurtosis
- Skewness
- Shannon Entropy
- Activity
- Mobility
- Complexity
- Permutation Entropy
- Sample Entropy
- Approximate Entropy

In [11]:
# Hyperparams

window_length = 32


In [12]:
# %%time
# class NeonatalSeizureFeatures:
#     def __init__(self, row):
#         self.row = row
        
#     def skewness(self):
#         row = np.array(self.row)
#         row = row[:-1]
#         row = np.reshape(row, (21, window_length))
#         return (pd.Series(scipy.stats.skew(x, axis = 0, bias = False) for x in row))
    
    
# df_new = df.apply(lambda row: NeonatalSeizureFeatures(row).skewness(), axis = 1)

# df_new

### Helper Methods:

In [13]:
def hMob(x):
    row = np.array(x)
    return (np.sqrt(np.var(np.gradient(x)) / np.var(x)))

### Feature Methods:

In [28]:
# Feature Methods

def feature_mean(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(np.mean(x, axis = 0) for x in row))

    
def feature_stddev(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(np.std(x, axis = 0) for x in row))

def kurtosis(row):
    row = np.array(row)
    annotation = row[-1]
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(scipy.stats.kurtosis(x, axis = 0, bias = False) for x in row))

def skewness(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(scipy.stats.skew(x, axis = 0, bias = False) for x in row))
    
def spectral_entropy(row, sf = 32, nperseg = window_length, axis = 1):
    row = np.array(row)
    annotation = row[-1]
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    _, psd = welch(row, sf, nperseg=nperseg, axis=axis)
    psd_norm = psd / psd.sum(axis=axis, keepdims=True)
    se = - np.where(psd_norm == 0, 0, psd_norm * np.log(psd_norm) / np.log(2)).sum(axis=axis)
    return pd.Series(se)

def hjorthActivity(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(np.var(x, axis = 0) for x in row))

def hjorthMobility(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(np.sqrt(np.var(np.gradient(x)) / np.var(x)) for x in row))

def hjorthComplexity(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series((hMob(np.gradient(x)) / hMob(x)) for x in row))

def permutation_entropy(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(ant.perm_entropy(x) for x in row))

def sample_entropy(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(ant.sample_entropy(x) for x in row))

def approximate_entropy(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(ant.app_entropy(x) for x in row))

In [29]:
list_of_feature_methods = [feature_mean, feature_stddev, kurtosis, skewness, spectral_entropy, hjorthActivity, hjorthMobility, 
                          hjorthComplexity, permutation_entropy, sample_entropy, approximate_entropy]

In [30]:
%%time
df_list = list()
for i, j in zip(list_of_feature_methods, range(len(list_of_feature_methods))):
    print("Epoch %d ..." % (j+1))
    df_temp = df.apply(lambda row: i(row), axis = 1)
    df_list.append(df_temp)

new_df = pd.concat(df_list, axis = 1)
new_df

Epoch 1 ...


AxisError: axis 0 is out of bounds for array of dimension 0

In [13]:
feature_df = pd.concat([new_df, df[df.columns[-1]]], axis = 1)
feature_df

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,672
0,0.705170,0.627579,0.563296,0.561497,0.661709,0.500225,0.714688,0.464326,0.471334,0.412236,...,0.207685,0.146087,0.136807,0.331072,0.180035,0.099877,0.092088,0.454638,0.102746,0.0
1,0.573662,0.507454,0.542473,0.483116,0.511353,0.437124,0.533494,0.495543,0.399331,0.550337,...,0.056649,0.101368,0.277073,0.190806,0.281063,0.233324,0.429936,0.213984,0.145052,0.0
2,0.636943,0.520640,0.488823,0.529611,0.541663,0.511655,0.665044,0.549172,0.471088,0.532921,...,0.216965,0.101368,0.101368,0.241633,0.118247,0.181526,0.216965,0.348141,0.185484,0.0
3,0.484862,0.562332,0.611532,0.441587,0.489014,0.396626,0.460126,0.454295,0.441191,0.557765,...,0.190806,0.190806,0.184780,0.168446,0.120074,0.281881,0.315683,0.485606,0.187477,0.0
4,0.538846,0.536673,0.568350,0.550591,0.624571,0.474852,0.563601,0.410797,0.557898,0.472718,...,0.072037,0.198405,0.477990,0.232353,0.148396,0.180035,0.367465,0.367984,0.169919,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383266,0.384066,0.493491,0.404637,0.596175,0.419632,0.378368,0.454108,0.283672,0.364143,0.408382,...,0.116756,0.189315,0.271782,0.250914,0.149886,0.126036,0.101368,0.195859,0.136807,0.0
383267,0.440657,0.546737,0.481236,0.497694,0.578748,0.415171,0.500591,0.510396,0.543095,0.438918,...,0.090597,0.241633,0.213984,0.257212,0.267793,0.191624,0.146087,0.205150,0.306403,0.0
383268,0.345382,0.472884,0.345414,0.476114,0.349591,0.397218,0.423759,0.405204,0.551230,0.402841,...,0.362711,0.294278,0.090597,0.158494,0.146087,0.535182,0.101368,0.191650,0.191981,0.0
383269,0.503639,0.457108,0.522277,0.459348,0.463334,0.553888,0.471511,0.712468,0.583607,0.610103,...,0.237834,0.190806,0.295633,0.383390,0.181526,0.298978,0.101368,0.220179,0.248359,0.0


In [14]:
feature_df.to_csv('Full_feature_data1sec.csv', index = False)

In [118]:
feature_df1.columns = [i for i in range(232)]
feature_df1.columns

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            222, 223, 224, 225, 226, 227, 228, 229, 230, 231],
           dtype='int64', length=232)

In [114]:
np.isinf(feature_df1).values.any()

False

In [112]:
feature_df1 = feature_df.replace([np.inf, -np.inf], np.nan)
feature_df1.dropna(inplace = True)

In [116]:
feature_df1.reset_index(drop = True, inplace = True)

## Principal Component Analysis (PCA):

In [15]:
# Imports

from sklearn.decomposition import PCA

In [16]:
# Set hyperparams for PCA

n_components = 20
random_state = 32

### PCA 20

In [119]:
pca_20 = PCA(n_components = n_components, random_state = random_state)
feature_df_20 = pca_20.fit_transform(feature_df1[feature_df1.columns[:-1]])
# feature_df_20 = pca_20.fit_transform(feature_df1[feature_df1[[-1]]])

In [120]:
pca_20_df = pd.DataFrame(data = feature_df_20)
pca_20_df = pd.concat([pca_20_df, feature_df1[feature_df1.columns[-1]]], axis = 1)
pca_20_df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,231
0,4.901421,2.701643,-1.275170,5.143429,-1.620967,0.613150,1.286259,-1.417769,1.444722,0.600224,...,0.060839,1.288825,-1.794129,0.458089,-1.073227,0.326684,-1.218720,-0.406938,-2.917282,0.0
1,0.863469,-5.155784,19.391457,3.268594,4.865219,0.716133,1.389465,-2.069688,-0.711067,-0.462562,...,-1.037438,-0.175020,-0.961911,0.866116,0.116357,-0.843982,-0.674588,-0.588865,0.109306,0.0
2,-6.782143,2.158658,2.189458,0.071391,1.237314,0.105731,0.060489,-0.475559,-0.271702,-0.060264,...,-0.058390,0.020884,-0.124925,0.076326,0.022127,-0.099033,-0.052237,-0.094348,0.041938,0.0
3,-7.053188,2.439245,1.538124,-0.132499,-0.224322,-0.010770,0.068195,-0.152156,-0.053641,-0.049488,...,-0.070432,-0.007473,-0.035539,-0.013059,-0.008530,-0.138042,-0.035067,-0.041577,-0.008063,0.0
4,-7.535043,3.178896,0.499794,-0.285159,-0.245198,-0.032130,0.100292,-0.220618,-0.253111,-0.046353,...,-0.073012,-0.005609,-0.119076,0.154522,0.022232,-0.011616,-0.018695,-0.026029,0.018659,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16291,2.121631,0.114495,-3.814118,1.318359,0.716929,-0.409302,3.169799,0.978609,0.101912,1.310513,...,0.066802,0.060389,1.356242,4.030198,-2.779573,0.908256,0.892026,1.440066,1.452993,0.0
16292,7.051724,0.046287,-3.450367,-5.499100,3.477877,0.270105,1.225670,-1.294833,-1.269587,-0.139015,...,-0.934864,0.667068,0.386159,0.555513,-1.243616,0.222214,0.239342,-0.141758,-0.135718,0.0
16293,-3.022786,-3.315476,-3.797521,-0.893076,0.438185,0.728091,-0.032998,-0.376513,1.373492,0.412405,...,0.973510,0.179913,-0.635228,1.117988,0.627298,-0.204403,0.180860,0.707815,0.536029,0.0
16294,5.998321,1.029297,-4.115128,-4.276564,-0.218342,-0.376820,2.220890,-1.687479,-0.395006,0.691122,...,-0.916872,0.898624,0.312184,2.311207,-0.437540,-0.726567,-0.122660,-1.774690,0.921198,0.0


In [121]:
pca_20_df.to_csv('1sec/PCA_20_features.csv', index = False)

### PCA 50

In [122]:
n_components = 50

pca_50 = PCA(n_components = n_components, random_state = random_state)
feature_df_50 = pca_50.fit_transform(feature_df1[feature_df1.columns[:-1]])

pca_50_df = pd.DataFrame(data = feature_df_50)
pca_50_df = pd.concat([pca_50_df, feature_df1[feature_df1.columns[-1]]], axis = 1)
pca_50_df

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,231
0,4.901421,2.701643,-1.275170,5.143429,-1.620970,0.613131,1.286226,-1.417766,1.444427,0.600575,...,0.570364,-0.886126,-0.982307,-0.496790,-0.022562,-0.273687,-0.463349,-0.050930,0.801801,0.0
1,0.863469,-5.155784,19.391457,3.268593,4.865218,0.716138,1.389489,-2.069694,-0.710727,-0.461043,...,0.200579,-0.153199,-0.368352,-0.341452,-0.129752,-0.302006,-0.181581,0.478378,0.159127,0.0
2,-6.782143,2.158658,2.189458,0.071391,1.237313,0.105730,0.060494,-0.475554,-0.271635,-0.059872,...,0.153351,-0.044525,-0.039783,0.017995,-0.005723,0.014173,0.013893,-0.093317,-0.037416,0.0
3,-7.053188,2.439245,1.538124,-0.132499,-0.224322,-0.010769,0.068199,-0.152157,-0.053590,-0.049310,...,0.131352,-0.069443,-0.152969,0.032218,0.066243,0.084304,-0.032498,-0.099924,0.058492,0.0
4,-7.535043,3.178896,0.499794,-0.285159,-0.245198,-0.032130,0.100292,-0.220618,-0.253105,-0.046304,...,-0.002161,-0.006512,0.016940,0.006546,0.002581,-0.020468,0.001606,-0.005654,0.003170,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16291,2.121631,0.114495,-3.814118,1.318360,0.716924,-0.409301,3.169799,0.978602,0.101853,1.310425,...,-1.338026,-0.512642,0.503093,-0.327117,0.025187,-0.431047,0.222354,0.452380,-0.353413,0.0
16292,7.051724,0.046287,-3.450367,-5.499101,3.477877,0.270105,1.225675,-1.294856,-1.269620,-0.138700,...,-0.428840,0.173946,-0.462803,0.143407,-0.377900,-0.379543,0.331191,0.161700,-0.224093,0.0
16293,-3.022786,-3.315476,-3.797521,-0.893076,0.438187,0.728078,-0.033028,-0.376491,1.373422,0.413041,...,-0.180644,-0.363027,0.291971,-0.138437,-0.195820,-0.505695,0.029818,0.249897,-0.098363,0.0
16294,5.998321,1.029297,-4.115128,-4.276564,-0.218340,-0.376820,2.220898,-1.687488,-0.394971,0.691774,...,0.198149,0.404801,0.529178,0.758978,0.010270,-0.992147,0.239563,0.222264,-0.214098,0.0


In [123]:
pca_50_df.to_csv('1sec/PCA_50_features.csv', index = False)

### PCA 70

In [124]:
n_components = 70

pca_70 = PCA(n_components = n_components, random_state = random_state)
feature_df_70 = pca_70.fit_transform(feature_df1[feature_df1.columns[:-1]])

pca_70_df = pd.DataFrame(data = feature_df_70)
pca_70_df = pd.concat([pca_70_df, feature_df1[feature_df1.columns[-1]]], axis = 1)

pca_70_df.to_csv('1sec/PCA_70_features.csv', index = False)

### PCA 100

In [126]:
n_components = 100

pca_100 = PCA(n_components = n_components, random_state = random_state)
feature_df_100 = pca_100.fit_transform(feature_df1[feature_df1.columns[:-1]])

pca_100_df = pd.DataFrame(data = feature_df_100)
pca_100_df = pd.concat([pca_100_df, feature_df1[feature_df1.columns[-1]]], axis = 1)

pca_100_df.to_csv('1sec/PCA_100_features.csv', index = False)